In [29]:
import time

In [30]:
start = time.time()
import cv2

In [31]:
# Load the cascade
face_cascade = cv2.CascadeClassifier('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\haarcascade_frontalface_default.xml')

In [32]:
# Read the input image
img = cv2.imread('.\\FinalTestLogistic\\test.png')
img2 = cv2.imread('.\\FinalTestLogistic\\test.png')

In [33]:
# Convert into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [34]:
# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)

In [35]:
# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(img2, (x, y), (x+w, y+h), (255, 0, 0), 2)

In [36]:
# Display the output
cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\FinalTestLogistic\\face detection result.png', img2)

True

In [37]:
from PIL import Image
import numpy as np
import os
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import random
from math import log
from tqdm import tqdm

In [38]:
def getImageMatrix(imageName):
    im = Image.open(imageName, mode="r", formats=None) 
    pix = im.load()
    color = 1
    if type(pix[0,0]) == int:
      color = 0
    image_size = im.size 
    image_matrix = []
    for width in range(int(image_size[0])):
        row = []
        for height in range(int(image_size[1])):
                row.append((pix[width,height]))
        image_matrix.append(row)
    return image_matrix,image_size[0],color

In [39]:
def LogisticEncryption(imageName, key):
    N = 256
    key_list = [ord(x) for x in key]
    G = [key_list[0:4] ,key_list[4:8], key_list[8:12]]
    g = []
    R = 1
    for i in range(1,4):
        s = 0
        for j in range(1,5):
            s += G[i-1][j-1] * (10**(-j))
        g.append(s)
        R = (R*s) % 1

    L = (R + key_list[12]/256) % 1
    S_x = round(((g[0]+g[1]+g[2])*(10**4) + L *(10**4)) % 256)
    V1 = sum(key_list)
    V2 = key_list[0]
    for i in range(1,13):
        V2 = V2 ^ key_list[i]
    V = V2/V1

    L_y = (V+key_list[12]/256) % 1
    S_y = round((V+V2+L_y*10**4) % 256)
    C1_0 = S_x
    C2_0 = S_y
    C = round((L*L_y*10**4) % 256)
    C_r = round((L*L_y*10**4) % 256)
    C_g = round((L*L_y*10**4) % 256)
    C_b = round((L*L_y*10**4) % 256)
    x = 4*(S_x)*(1-S_x)
    y = 4*(S_y)*(1-S_y)
    
    imageMatrix,dimension, color = getImageMatrix(imageName)
    LogisticEncryptionIm = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            while x <0.8 and x > 0.2 :
                x = 4*x*(1-x)
            while y <0.8 and y > 0.2 :
                y = 4*y*(1-y)
            x_round = round((x*(10**4))%256)
            y_round = round((y*(10**4))%256)
            C1 = x_round ^ ((key_list[0]+x_round) % N) ^ ((C1_0 + key_list[1])%N)
            C2 = x_round ^ ((key_list[2]+y_round) % N) ^ ((C2_0 + key_list[3])%N) 
            if color:
              C_r =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][0]) % N) ^ ((C_r + key_list[7]) % N)
              C_g =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][1]) % N) ^ ((C_g + key_list[7]) % N)
              C_b =((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j][2]) % N) ^ ((C_b + key_list[7]) % N)
              row.append((C_r,C_g,C_b))
              C = C_r

            else:
              C = ((key_list[4]+C1) % N) ^ ((key_list[5]+C2) % N) ^ ((key_list[6]+imageMatrix[i][j]) % N) ^ ((C + key_list[7]) % N)
              row.append(C)

            x = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            y = (x + C/256 + key_list[8]/256 + key_list[9]/256) % 1
            for ki in range(12):
                key_list[ki] = (key_list[ki] + key_list[12]) % 256
                key_list[12] = key_list[12] ^ key_list[ki]
        LogisticEncryptionIm.append(row)

    im = Image.new("L", (dimension, dimension))
    if color:
        im = Image.new("RGB", (dimension, dimension))
    else: 
        im = Image.new("L", (dimension, dimension)) # L is for Black and white pixels
      
    pix = im.load()
    for x in range(dimension):
        for y in range(dimension):
            pix[x, y] = LogisticEncryptionIm[x][y]
    im.save(imageName + "_LogisticEnc.png", "PNG")


In [40]:
#input params for arnold cat encryption function
image = ".\\FinalTestLogistic\\temp"
ext = ".bmp"
key = "abcdefghijklm"

In [41]:
for (x, y, w, h) in faces:
    cropped_image = img[y:y+h, x:x+w]
    #cv2.imshow("crop faces", cropped_image)
    cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\FinalTestLogistic\\temp.bmp', cropped_image)
    LogisticEncryption(image + ext, key)
    im = Image.open(image + ".bmp_LogisticEnc.PNG", 'r')
    #imshow(np.asarray(im))
    LogisticResult = cv2.imread(".\\FinalTestLogistic\\temp.bmp_LogisticEnc.png",cv2.IMREAD_COLOR)
    #print(HenonResult)
    #cv2.imshow('Henon Result', HenonResult)
    img[y:y+h, x:x+w] = LogisticResult
    #cv2.waitKey(0)
end = time.time()

In [42]:
print("The time of execution of Logistic Chaos Map is: ", (end-start) * 10**3, "ms")

The time of execution of Logistic Chaos Map is:  2756.058692932129 ms


In [43]:
cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\FinalTestLogistic\\Encryption Result.png', img)

True

### Henon Decryption

In [44]:
def HenonDecryption(imageNameEnc, key):
    imageMatrix, dimension, color = getImageMatrix(imageNameEnc)
    transformationMatrix = genHenonMap(dimension, key)
    pil_im = Image.open(imageNameEnc, 'r')
    imshow(np.asarray(pil_im))
    henonDecryptedImage = []
    for i in range(dimension):
        row = []
        for j in range(dimension):
            try:
                if color:
                    row.append(tuple([transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]]))
                else:
                    row.append(transformationMatrix[i][j] ^ imageMatrix[i][j])
            except:
                if color:
                    row = [tuple([transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]])]
                else :
                    row = [transformationMatrix[i][j] ^ x for x in imageMatrix[i][j]]
        try:
            henonDecryptedImage.append(row)
        except:
            henonDecryptedImage = [row]
    if color:
        im = Image.new("RGB", (dimension, dimension))
    else: 
        im = Image.new("L", (dimension, dimension)) # L is for Black and white pixels

    pix = im.load()
    for x in range(dimension):
        for y in range(dimension):
            pix[x, y] = henonDecryptedImage[x][y]
    im.save(imageNameEnc.split('_')[0] + "_HenonDec.png", "PNG")

In [45]:
#decryption test
imgd = cv2.imread('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\decrypt.jpg')

for (x, y, w, h) in faces:
    cropped_image = imgd[y:y+h, x:x+w]
    cv2.imwrite('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\test.jpg', cropped_image)
    c_image=cv2.imread('E:\\svnit\\Assignments\\SEM 7\\New FYP\\Attempt2\\test.jpg')
    ArnoldCatDecrypt = ArnoldCatDecryption(image + ext , key)
    imgd[y:y+h, x:x+w] = ArnoldCatDecrypt
    cv2.waitKey(0)

TypeError: 'NoneType' object is not subscriptable